# Figure 1 - Analysis

In [ ]:
import sys
sys.path.append("..")

from main import *

plt.rcParams['font.size'] = 18

# Simulating neuronal activity in ellipse geometry

First, we run an example simulation of neurons within the ellipsoid geometry.

In [ ]:
%matplotlib inline

In [ ]:
vertices = np.load('../Files/vertices_ellipse2.npy').astype('float')
eigenmodes = np.load('../Files/eigenmodes_ellipse2.npy')

ellipse = Geometry(vertices, eigenmodes)

In [ ]:
params = {'N_neurons': 2500,
          'h': 0.1,
          'g': 5,
          'tau': 3,
          'dale': False
          }

np.random.seed(420)
simulator = Simulator(ellipse,
                      ChaoticRNN,
                      params)

simulator.integrate(1000, output=False)

In [ ]:
%matplotlib inline
simulator.plot(spacing=1.5)

In [ ]:
%matplotlib inline
simulator.imshow(vmin=0, vmax=1)

The animated ellipsoid dynamics can be visualized here (requires `PyQt5`).

In [ ]:
%matplotlib qt
simulator.animate(alpha=0.75,
                  cmap='hot',
                  vmax=1.5)

Next, we average the firing rate correlations of neurons over 100 simulations. 

In [ ]:
C = simulator.compute_average_correlations(n_iters=100, T=1000)

In [ ]:
%matplotlib inline
plt.figure(figsize=(5, 5))
plt.imshow(C, cmap='Reds', vmin=0.1, vmax=0.2, interpolation=None)

Finally, we compute FC gradients and compare them to the geometric eigenmodes of the ellipsoid.

In [ ]:
mode_similarity, mapping, functional_gradients = simulator.compute_geometric_mapping(C, N_modes=20, return_gradients=True)

In [ ]:
%matplotlib inline
fig, ax = plt.subplots(figsize=(5, 5))
cax = plt.imshow(np.abs(mode_similarity), cmap='Reds')
plt.xlabel('Functional modes')
plt.ylabel('Geometric modes')
plt.colorbar(cax, ax=ax, fraction=0.045, pad=0.02)

#### Rotating gradients

Here, we're rotating the gradients to properly align the different eigengroups (eigenmodes of similar wavelengths with different rotations) with their corresponding eigenmodes, simply for visual purposes. Due to the rotational symmetry of the ellipsoid, along with discretization effects of subsampling coordinates, there is no guarantee that gradients are rotationally aligned with eigenmodes. In practice, however, we avoid using this rotation procedure throughout the rest of the paper as it only marginally improves the eigenmode-gradient correlations, and requires extra computation time. We did it here to obtain a proper alignment for the figure, but similar results may have been obtained simply by reinitializing the random seed and picking one where gradients align nicely with eigenmodes. In other asymmetrical geometries (i.e. heart, cow), this procedure becomes useless.

In [ ]:
coords, mode_similarity = find_optimal_rotation(simulator.geometry, functional_gradients)

In [ ]:
%matplotlib inline
fig, ax = plt.subplots(figsize=(5, 5))
cax = plt.imshow(np.abs(mode_similarity), cmap='Reds')
plt.xlabel('Functional modes')
plt.ylabel('Geometric modes')
plt.colorbar(cax, ax=ax, fraction=0.045, pad=0.02)

In [ ]:
save = False
if save:
    np.save('Files/figure1_timeseries.npy', simulator.timeseries)
    np.save('Files/figure1_correlation_matrix.npy', C)
    np.save('Files/figure1_mode_similarity.npy', mode_similarity)

# Plotting ellipse with local connections

Generating a `.png` image of locally connected neurons within the ellipsoid for Figure 1**b**. In general, 3D plots are annoying to work with and crop appropriately in multi-panel figures, so we opt to generate high-resolution `.png` images which we incorporate in the figures later.

In [ ]:
%matplotlib inline

In [ ]:
simulator.coordinates -= np.mean(simulator.coordinates, axis=0)

In [ ]:
W = np.abs(simulator.torch_to_numpy(simulator.dynamics.W))
coords = simulator.coordinates
coords += 0.5

In [ ]:
i = 250

neuron1 = 2500 - i
targets1 = np.where(W[neuron1] > 0)[0]

neuron2 = i
targets2 = np.where(W[neuron2] > 0)[0]

In [ ]:
red = plt.get_cmap('coolwarm')(250)
blue = plt.get_cmap('coolwarm')(0)
#red = 'red'
#blue = 'blue'

In [ ]:
fig, ax = plt.subplots(subplot_kw={"projection": "3d"}, figsize=(5, 5), dpi=300)

ax.scatter(coords[:, 0], coords[:, 1], coords[:, 2], color=[0.5, 0.5, 0.5], s=5, alpha=0.2, edgecolor='None', zorder=-10)
ax.set_xlim([0, 1])
ax.set_ylim([0, 1])
ax.set_zlim([0, 1])

ax.scatter(coords[neuron1, 0], coords[neuron1, 1], coords[neuron1, 2], color=red, s=25, alpha=1, edgecolor='None')
ax.scatter(coords[targets1, 0], coords[targets1, 1], coords[targets1, 2], color=red, s=5, alpha=0.25, edgecolor='None')
for target1 in targets1:
    x = [coords[neuron1, 0], coords[target1, 0]]
    y = [coords[neuron1, 1], coords[target1, 1]]
    z = [coords[neuron1, 2], coords[target1, 2]]
    ax.plot(x, y, z, color=red, linewidth=0.5, alpha=0.25)

ax.scatter(coords[neuron2, 0], coords[neuron2, 1], coords[neuron2, 2], color=blue, s=25, alpha=1, edgecolor='None')
ax.scatter(coords[targets2, 0], coords[targets2, 1], coords[targets2, 2], color=blue, s=5, alpha=0.25, edgecolor='None')
for target2 in targets2:
    x = [coords[neuron2, 0], coords[target2, 0]]
    y = [coords[neuron2, 1], coords[target2, 1]]
    z = [coords[neuron2, 2], coords[target2, 2]]
    ax.plot(x, y, z, color=blue, linewidth=0.5, alpha=0.25)
    
ax.set_axis_off()

plt.tight_layout(pad=0)
plt.savefig('Figures/figure1_ellipse.png')
plt.show()

Note that the random seed might highlight different neurons than what is shown in the paper.

# Plotting individual gradients

Next, we generate `.png` images of the FC gradients on the ellipse.

In [ ]:
%matplotlib inline

In [ ]:
for i in range(6):
    fig, ax = plt.subplots(subplot_kw={"projection": "3d"}, figsize=(5, 5), dpi=300)
    
    ax.scatter(coords[:, 0], coords[:, 1], coords[:, 2], c=np.sign(mode_similarity[i, i]) * functional_gradients[mapping[i]], alpha=0.5, cmap='coolwarm', edgecolor='None')
    ax.set_xlim([0, 1])
    ax.set_ylim([0, 1])
    ax.set_zlim([0, 1])
        
    ax.set_axis_off()
    
    plt.tight_layout(pad=0)
    plt.savefig('../Figures/figure1_gradient{}.png'.format(i))
    plt.show()


# Plotting individual eigenmodes

Next, we generate `.png` images of the eigenmodes on the ellipse.

In [ ]:
%matplotlib inline

In [ ]:
vertices -= np.mean(vertices, axis=0)
vertices += 0.5

In [ ]:
for i in range(6):
    fig, ax = plt.subplots(subplot_kw={"projection": "3d"}, figsize=(5, 5), dpi=300)
    
    ax.scatter(vertices[:, 0], vertices[:, 1], vertices[:, 2], c=eigenmodes[:, i + 1], alpha=0.5, cmap='coolwarm', edgecolor='None')
    ax.set_xlim([0, 1])
    ax.set_ylim([0, 1])
    ax.set_zlim([0, 1])
        
    ax.set_axis_off()
    
    plt.tight_layout(pad=0)
    plt.savefig('../Figures/figure1_eigenmodes{}.png'.format(i))
    plt.show()

# Generating Figure 1

Finally, we incorporate all the previous elements to generate Figure 1.

In [ ]:
from PIL import Image
from visualization import *

In [ ]:
R = np.load('Files/figure1_timeseries.npy')[:, :700] # Truncating for visualization
C = np.load('Files/figure1_correlation_matrix.npy')
mode_similarity = np.load('Files/figure1_mode_similarity.npy')

In [ ]:
fig = PaperFigure(figsize=(7, 5), dpi=600)

fig.set_tick_length(1)
fig.set_font_size(6)
fig.add_background()

# Adding panels -------------------------------

fig.add_axes('ellipse1', (1.25, 0), 1.125, 1.5)
fig.add_axes('timeseries', (2.75, 0), 2.25, 1.5)
fig.add_axes('correlations', (5.5, 0), 1.5, 1.5)

y = 1.8
w = 0.65
pad = (5 - 6 * w) / 5
ratio = 1.6
for i in range(6):
    fig.add_axes(f'gradient{i+1}', (i * (w + pad), y), w, ratio * w)
    fig.add_axes(f'mode{i+1}', (i * (w + pad), y + ratio * w), w, ratio * w)

fig.add_axes('curve', (5.5, y), 1.5, 0.5)
fig.add_axes('correlations_modes', (5.5, y + 0.6), 1.5, 1.5)

fig.set_line_thickness(0.6)

# Content -------------------------------------

ax = fig.axes['ellipse1']
image = np.array(Image.open('Figures/figure1_ellipse.png'))[:, :, :3]
ax.imshow(image, aspect='auto')
center = 730
ax.set_xlim([center - 250, center + 250])
ax.set_ylim([center - 350, center + 300])
ax.set_xticks([])
ax.set_yticks([])
ax.axis('off')

ax = fig.axes['timeseries']
for i in range(25):
    if i == 0:
        maxval = np.max(R[i])
    if i == 24:
        minval = np.min(R[i] - 3 * i)
    ax.plot(R[i] - 3 * i, color='black', linewidth=0.4)
ax.set_xlim([0, R.shape[1]])
ax.set_xticks([])
ax.set_yticks([])
ax.set_ylim(minval - 0.25, maxval + 0.25)
ax.axis('off')

ax = fig.axes['correlations']
ax.imshow(C, cmap='Reds', vmin=0.05, vmax=0.2)
ax.set_xticks([])
ax.set_yticks([])

ax = fig.axes['curve']
y = np.abs(np.diag(mode_similarity))
#for i in range(20):
#    ax.scatter(i, y[i], color='black', s=2)
#    ax.plot([i, i], [0, y[i]], color='black', linewidth=1)
x = np.arange(20)
ax.bar(x[y > 0.5], y[y > 0.5], color='white', edgecolor='black', linewidth=0.5, width=1)
ax.bar(x[y < 0.5], y[y < 0.5], color=[1, 0.75, 0.75], edgecolor='black', linewidth=0.5, width=1)

ax.plot([-10, 30], [0.5, 0.5], color='black', linewidth=0.5, linestyle='--')
ax.set_xticks([])
ax.set_ylim([0, 1.1])
ax.spines[['top', 'right']].set_visible(False)
ax.set_xlim([-0.5, 19.5])

ax = fig.axes['correlations_modes']
ax.imshow(np.abs(mode_similarity), cmap='Reds', vmin=0, vmax=1)
ax.set_xticks([])
ax.set_yticks([])

keys = ['gradient1', 'gradient2', 'gradient3', 'gradient4', 'gradient5', 'gradient6']
for i, key in enumerate(keys):
    ax = fig.axes[key]
    image = np.array(Image.open('Figures/figure1_gradient{}.png'.format(i)))[:, :, :3]
    ax.imshow(image, aspect='auto')
    center = 770
    ax.set_xlim([center - 250, center + 250])
    ax.set_ylim([center + 400, center - 370])
    ax.set_xticks([])
    ax.set_yticks([])
    ax.axis('off')

keys = ['mode1', 'mode2', 'mode3', 'mode4', 'mode5', 'mode6']
for i, key in enumerate(keys):
    ax = fig.axes[key]
    image = np.array(Image.open('Figures/figure1_eigenmodes{}.png'.format(i)))[:, :, :3]
    ax.imshow(image, aspect='auto')
    center = 770
    ax.set_xlim([center - 250, center + 250])
    ax.set_ylim([center + 320, center - 450])
    ax.set_xticks([])
    ax.set_yticks([])
    ax.axis('off')

# Displaying ----------------------------------

fig.show()

In [ ]:
fig.save('../Figures/figure1_incomplete.svg')

Labels are manually added in `Inkscape`.